In [1]:
import numpy as np
import os
import torch 
import yaml 

new_directory = '/home/franciscoperez/Documents/GitHub/CNN-PELSVAE2/cnn-pels-vae/'
os.chdir(new_directory)

from src.utils import get_time_sequence, get_time_from_period

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
                      
with open('src/configuration/nn_config.yaml', 'r') as file:
    nn_config = yaml.safe_load(file)

In [6]:
def revert_light_curve(period, folded_normed_light_curve, 
                        original_sequences, faintness=1.0, 
                        classes = None, noise=True):
    """
    Revert previously folded and normed light curves back to the original light curves.

    Parameters:
        period (float): The period of the variability in the light curves.
        folded_normed_light_curve (numpy.ndarray): A 3D array representing the folded and normed light curves.
        faintness (float, optional): A scaling factor to adjust the magnitude values of the reverted light curves.
                                     Defaults to 1.0, meaning no scaling.

    Returns:
        list: A list of 2D arrays representing the reverted real light curves with time and magnitude values.
    """
    num_sequences = folded_normed_light_curve.shape[0]
    reverted_light_curves = []
    magnitude_min_max = get_time_sequence(n=1, star_class=classes)

    for i in range(num_sequences):
        # Extract the time (period) and magnitude values from the folded and normed light curve
        time = original_sequences[i] #folded_normed_light_curve[i,:,0]
        if np.max(folded_normed_light_curve[i,:,0])<0.95: 
            continue

        normed_magnitudes_min = np.min(folded_normed_light_curve[i,:,1])
        normed_magnitudes_max = np.max(folded_normed_light_curve[i,:,1])
        
        normed_magnitudes = ((folded_normed_light_curve[i,:,1]-normed_magnitudes_min)/
                            (normed_magnitudes_max-normed_magnitudes_min))

        # Generate the time values for the reverted light curve
        [original_min, original_max, error_mean, error_std] = magnitude_min_max[i]

        #real_time =  time #get_time_from_period(period[i], time, example_sequence, sequence_length=600)

        real_time =  get_time_from_period(period[i],  folded_normed_light_curve[i,:,0], time, sequence_length=600)

        # Revert the normed magnitudes back to the original magnitudes using min-max scaling and faintness factor
        if noise:
            original_magnitudes = ((normed_magnitudes * (original_max - original_min)) + original_min) * faintness
            random_noise = np.random.normal(error_mean, error_std, original_magnitudes.shape)
            original_magnitudes += random_noise

        else: 
            original_magnitudes = ((normed_magnitudes * (original_max - original_min)) + original_min) * faintness

        if isinstance(real_time, torch.Tensor):
            if real_time.is_cuda:
                real_time = real_time.cpu().numpy()
            else:
                real_time = real_time.numpy()

        # Convert original_magnitudes to NumPy array if it's a PyTorch tensor
        if isinstance(original_magnitudes, torch.Tensor):
            if original_magnitudes.is_cuda:
                original_magnitudes = original_magnitudes.cpu().numpy()
            else:
                original_magnitudes = original_magnitudes.numpy()

        # Now, you can use np.column_stack without issues
        reverted_light_curve = np.column_stack((original_magnitudes, real_time))

        reverted_light_curves.append(reverted_light_curve)
    
    reverted_light_curves = np.stack(reverted_light_curves)

    reverted_light_curves = np.swapaxes(reverted_light_curves, 1, 2)
    # Generate random unique indices along the last dimension
    random_indices = np.random.choice(350, nn_config['data']['seq_length']+1, replace=False)

    # Sort the indices for easier interpretation and debugging (optional)
    random_indices = random_indices.sort()

    # Select 200 random observations
    reverted_light_curves_random = reverted_light_curves[:, :, random_indices]

    reverted_light_curves_random = reverted_light_curves_random.squeeze(2) 

    for i in range(reverted_light_curves_random.shape[0]):
        sort_indices = np.argsort(reverted_light_curves_random[i, 1, :])
        for j in range(reverted_light_curves_random.shape[1]):
            reverted_light_curves_random[i, j, :] = reverted_light_curves_random[i, j, sort_indices]

    for i in range(reverted_light_curves_random.shape[0]):
        reverted_light_curves_random[i] = np.flipud(reverted_light_curves_random[i])

    print('Shape of reverted_light_curves_random[i, :, :]:', reverted_light_curves_random[i, :, :].shape)
    print('Shape of sort_indices:', sort_indices.shape)

    return reverted_light_curves_random

In [7]:
folded_normed_light_curve = np.random.rand(1, 600, 2)*10
period =  torch.from_numpy(np.asarray([1.0])).to(device) 
original_sequences = [[1.0, 30.0]]
classes = ['RRLYR']

In [8]:
revert_light_curve(period, folded_normed_light_curve, original_sequences, faintness=1.0, classes = classes)

/home/franciscoperez/Documents/GitHub/CNN-PELSVAE2/cnn-pels-vae/src/utils.py:862: FutureWarning: The default value of regex will change from True to False in a future version.
  lc_train['NUMBER'] = lc_train['NUMBER'].str.replace('.dat', '')
Getting time sequences: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.87it/s]

tensor(1., device='cuda:0')
Shape of reverted_light_curves_random[i, :, :]: (2, 600)
Shape of sort_indices: (600,)


array([[[  5.03614664,   5.95350933,   6.68607569, ..., 103.29819489,
         104.29350281, 105.11094666],
        [ 18.86968392,  19.60363921,  19.06058161, ...,  19.71860749,
          19.62448385,  19.10599356]]])